In [221]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torch import nn

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
# Import from https://github.com/vincrichard/UnsupervisedAnomalyDetection_Timeseries repo
from datetime import datetime

In [222]:
healthy = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 26, 27, 28, 29, 30]

In [227]:
import pathlib

directory = pathlib.Path('/kaggle/input/d/danilmatienko/all-datasets')
files = [str(f).replace("\\", "/") for f in directory.iterdir() if f.is_file()]


In [231]:
def to_sequences(x, y, seq_size=1):
    x_values = []
    y_values = []

    for i in range(len(x)-seq_size):
        x_values.append(x.iloc[i:(i+seq_size)].values)
        y_values.append(y.iloc[i+seq_size])
        
    return np.array(x_values), np.array(y_values)

In [254]:
import keras

In [301]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dropout, RepeatVector, TimeDistributed, Dense
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import Model
import seaborn as sns



# Define the model
model = Sequential()
model.add(LSTM(128, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dropout(rate=0.2))

model.add(RepeatVector(trainX.shape[1]))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(TimeDistributed(Dense(trainX.shape[2])))

# Compile the model
model.compile(optimizer='adam', loss='mae')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [261]:
check = keras.models.load_model("/kaggle/input/ggg/keras/model_top/1/my_model.keras")

In [ ]:
for file in sorted(files)[:30:2]:
    id = file.split("Np")[1].split("Nr")[0]
    if int(id) in healthy:
        path = file
        train = pd.read_csv(path)
        test = pd.read_csv(path.replace("1N-1", "2N-2", 1))

        target = '0'

        scaler = StandardScaler()
        train[target] = scaler.fit_transform(train[[target]])
        test[target] = scaler.transform(test[[target]])

        seq_size = 30  # Number of time steps to look back

        trainX, trainY = to_sequences(train[[target]], train[target], seq_size)
        testX, testY = to_sequences(test[[target]], test[target], seq_size)

        history = model.fit(trainX, trainX, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

        trainPredict = model.predict(trainX)
        trainMAE = np.mean(np.abs(trainPredict - trainX), axis=1)
        #plt.hist(trainMAE, bins=30)
        max_trainMAE = 0.5  #or Define 90% value of max as threshold.

        testPredict = model.predict(testX)
        testMAE = np.mean(np.abs(testPredict - testX), axis=1)
        #plt.hist(testMAE, bins=30)



In [240]:
model.save("my_model.keras")

In [203]:
anomaly_diff_dict = {}

In [218]:
def find_anomaly_interval(time):    
    time_answer = []
    i = 0
    time = list(time)
    while i < len(time) - 1:
        j = i
        i_start = i
        while (time[j+1] - time[j]) == 1 and j < len(time) - 2:
            j += 1
            i += 1
        time_answer.append((time[i_start], time[j]))
        i += 1
            
    return time_answer

In [264]:
for file in sorted(files)[::2]:
    id = file.split("Np")[1].split("Nr")[0]
    path = file
    
    target = '0'
    file = '/kaggle/input//d/danilmatienko/all-datasets/studyhakatondatasetsNp16Nr1N-1.csv'
    train = pd.read_csv(file)

    display(train)
    scaler = StandardScaler()
    train[target] = scaler.fit_transform(train[[target]])

    seq_size = 30  # Number of time steps to look back

    trainX, trainY = to_sequences(train[[target]], train[target], seq_size)
    display(trainX)
    trainPredict = check.predict(trainX)
    trainMAE = np.mean(np.abs(trainPredict - trainX), axis=1)
    max_trainMAE = 0.5  #or Define 90% value of max as threshold.

    
    print(trainPredict.shape)
    print(trainPredict)
    #Capture all details in a DataFrame for easy plotting
    anomaly_df = pd.DataFrame(test[30:])
    anomaly_df['testMAE'] = testMAE
    anomaly_df['max_trainMAE'] = max_trainMAE
    anomaly_df['anomaly'] = anomaly_df['testMAE'] > anomaly_df['max_trainMAE']
    anomaly_df[target] = test[target]
    anomaly_df['time'] = anomaly_df.index
    #Plot testMAE vs max_trainMAE
    # sns.lineplot(x=anomaly_df['Date'], y=anomaly_df['testMAE'])
    # sns.lineplot(x=anomaly_df['Date'], y=anomaly_df['max_trainMAE'])

    anomalies = anomaly_df.loc[anomaly_df['anomaly'] == True]
    anomaly_intervals = find_anomaly_interval(anomalies['time'])
    #print(anomaly_intervals)
    
    break
    

,Unnamed: 0,0
0,0,-5.262694e-05
1,1,-4.184267e-05
2,2,9.908224e-06
3,3,-8.775924e-06
4,4,-7.008343e-07
...,...,...
28855,28855,5.244338e-08
28856,28856,-6.734207e-07
28857,28857,-6.337306e-06
28858,28858,-1.002145e-05


array([[[-4.51382346],
        [-3.59232064],
        [ 0.82973186],
        ...,
        [-1.11685119],
        [-0.10664048],
        [ 0.40309887]],

       [[-3.59232064],
        [ 0.82973186],
        [-0.76680641],
        ...,
        [-0.10664048],
        [ 0.40309887],
        [-0.10562202]],

       [[ 0.82973186],
        [-0.76680641],
        [-0.0767996 ],
        ...,
        [ 0.40309887],
        [-0.10562202],
        [ 0.05274855]],

       ...,

       [[ 0.33262142],
        [-0.01090522],
        [ 0.04062887],
        ...,
        [ 0.79999282],
        [-0.01243291],
        [-0.07445714]],

       [[-0.01090522],
        [ 0.04062887],
        [-0.06386515],
        ...,
        [-0.01243291],
        [-0.07445714],
        [-0.55842944]],

       [[ 0.04062887],
        [-0.06386515],
        [-0.10633494],
        ...,
        [-0.07445714],
        [-0.55842944],
        [-0.8732355 ]]])

901/901 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
(28830, 30, 1)
[[[-4.0457001e+00]
  [-3.6568186e+00]
  [ 7.4446380e-01]
  ...
  [-9.6168536e-01]
  [-7.7740319e-02]
  [ 4.1073921e-01]]

 [[-3.3048511e+00]
  [ 7.1212655e-01]
  [-6.1029476e-01]
  ...
  [-7.3809445e-02]
  [ 3.8760903e-01]
  [-8.5545637e-02]]

 [[ 7.4919575e-01]
  [-5.5092329e-01]
  [-5.0236050e-02]
  ...
  [ 3.6354175e-01]
  [-6.5124005e-02]
  [ 8.9232214e-02]]

 ...

 [[ 3.1399021e-01]
  [ 5.5899400e-02]
  [ 5.1593684e-02]
  ...
  [ 7.1175385e-01]
  [ 1.8016089e-02]
  [-3.9133351e-02]]

 [[-1.5076825e-02]
  [ 7.8709118e-02]
  [-1.9584149e-02]
  ...
  [ 3.0705356e-03]
  [-4.2924799e-02]
  [-5.0016272e-01]]

 [[ 6.3208312e-02]
  [-1.9123888e-02]
  [-7.1817294e-02]
  ...
  [-5.0790202e-02]
  [-4.8481977e-01]
  [-7.9471058e-01]]]


In [251]:
for file in sorted(files)[::2]:
    id = file.split("Np")[1].split("Nr")[0]
    path = file
    train = pd.read_csv(path)
    test = pd.read_csv(path.replace("1N-1", "2N-2", 1))
    
    target = '0'

    scaler = StandardScaler()
    train[target] = scaler.fit_transform(train[[target]])

    seq_size = 30  # Number of time steps to look back

    trainX, trainY = to_sequences(train[[target]], train[target], seq_size)
    testX, testY = to_sequences(test[[target]], test[target], seq_size)
    trainPredict = model.predict(trainX)
    trainMAE = np.mean(np.abs(trainPredict - trainX), axis=1)
    max_trainMAE = 0.5  #or Define 90% value of max as threshold.

    testPredict = model.predict(testX)
    testMAE = np.mean(np.abs(testPredict - testX), axis=1)
    
    #Capture all details in a DataFrame for easy plotting
    anomaly_df = pd.DataFrame(test[30:])
    anomaly_df['testMAE'] = testMAE
    anomaly_df['max_trainMAE'] = max_trainMAE
    anomaly_df['anomaly'] = anomaly_df['testMAE'] > anomaly_df['max_trainMAE']
    anomaly_df[target] = test[target]
    anomaly_df['time'] = anomaly_df.index

    anomalies = anomaly_df.loc[anomaly_df['anomaly'] == True]
    anomaly_intervals = find_anomaly_interval(anomalies['time'])
    break
    

,Unnamed: 0,0
0,0,-5.262694e-05
1,1,-4.184267e-05
2,2,9.908224e-06
3,3,-8.775924e-06
4,4,-7.008343e-07
...,...,...
28855,28855,5.244338e-08
28856,28856,-6.734207e-07
28857,28857,-6.337306e-06
28858,28858,-1.002145e-05


array([[[-4.51382346],
        [-3.59232064],
        [ 0.82973186],
        ...,
        [-1.11685119],
        [-0.10664048],
        [ 0.40309887]],

       [[-3.59232064],
        [ 0.82973186],
        [-0.76680641],
        ...,
        [-0.10664048],
        [ 0.40309887],
        [-0.10562202]],

       [[ 0.82973186],
        [-0.76680641],
        [-0.0767996 ],
        ...,
        [ 0.40309887],
        [-0.10562202],
        [ 0.05274855]],

       ...,

       [[ 0.33262142],
        [-0.01090522],
        [ 0.04062887],
        ...,
        [ 0.79999282],
        [-0.01243291],
        [-0.07445714]],

       [[-0.01090522],
        [ 0.04062887],
        [-0.06386515],
        ...,
        [-0.01243291],
        [-0.07445714],
        [-0.55842944]],

       [[ 0.04062887],
        [-0.06386515],
        [-0.10633494],
        ...,
        [-0.07445714],
        [-0.55842944],
        [-0.8732355 ]]])

901/901 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
910/910 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
[[[ 0.00037397]
  [ 0.00051663]
  [ 0.00060287]
  ...
  [ 0.00375767]
  [ 0.00377222]
  [ 0.00378401]]

 [[-0.0002685 ]
  [-0.00052794]
  [-0.00069059]
  ...
  [ 0.00230449]
  [ 0.00231973]
  [ 0.00233202]]

 [[-0.00100275]
  [-0.00170332]
  [-0.00212866]
  ...
  [ 0.00056002]
  [ 0.00057524]
  [ 0.00058753]]

 ...

 [[-0.0036081 ]
  [-0.00604561]
  [-0.00756878]
  ...
  [-0.0024435 ]
  [-0.00240796]
  [-0.00237932]]

 [[-0.00225209]
  [-0.00389888]
  [-0.00499002]
  ...
  [-0.00032603]
  [-0.00029382]
  [-0.0002681 ]]

 [[-0.00081683]
  [-0.00164   ]
  [-0.00227839]
  ...
  [ 0.00247999]
  [ 0.00251102]
  [ 0.00253554]]]


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras

from sklearn.preprocessing import StandardScaler

def find_anomaly_interval(time):    
    time_answer = []
    i = 0
    time = list(time)
    while i < len(time) - 1:
        j = i
        i_start = i
        while (time[j+1] - time[j]) == 1 and j < len(time) - 2:
            j += 1
            i += 1
        time_answer.append((time[i_start], time[j]))
        i += 1
            
    return time_answer

def window_average(numbers, window_size, step_size):
    return pd.DataFrame([np.mean(numbers[i:i+window_size]) for i in range(0, len(numbers)-window_size+1, step_size)], columns=['value'])

def to_seq(x, y, seq_size=1):
    x_values = [] 
    y_values = []
    
    for i in range(len(x) - seq_size):
        x_values.append(x.iloc[i: (i + seq_size)].values)
        y_values.append(y.iloc[i + seq_size])
        
    return np.array(x_values), np.array(y_values)


def predict_result(data, path_to_model):
    
    scaler = StandardScaler()

    channel = 0
    signal_in_channel = data
    
    signal_in_channel = np.array(signal_in_channel).reshape(-1, 1)
    scaler.fit_transform(signal_in_channel)
    
    signal_in_channel = pd.DataFrame({'timestamp': np.arange(len(signal_in_channel)), 'value': signal_in_channel.reshape(-1)})
    
    signal_with_mean_transform = window_average(signal_in_channel['value'], 200, 200)
    
    
    seq_size = 30
    signal_croped_in_windows, _ = to_seq(
        signal_with_mean_transform[['value']], 
        signal_with_mean_transform['value'], 
        seq_size=seq_size
    )

    reconstructed_model = keras.models.load_model(path_to_model)

    output = reconstructed_model(signal_croped_in_windows)
    max_trainMAE = 0.1  #or Define 90% value of max as threshold.

    testMAE = np.mean(np.abs(output - signal_croped_in_windows), axis=1)
    print("testMAE:", testMAE)
    
    test = signal_with_mean_transform
    #print("TEST:", test)
    target = 'Fp1-M2'
    #Capture all details in a DataFrame for easy plotting
    anomaly_df = pd.DataFrame(test[30:])
    anomaly_df['testMAE'] = testMAE
    anomaly_df['max_trainMAE'] = max_trainMAE
    anomaly_df['anomaly'] = anomaly_df['testMAE'] > anomaly_df['max_trainMAE']
    anomaly_df[target] = test['value']
    anomaly_df['time'] = anomaly_df.index
    
    anomalies = anomaly_df.loc[anomaly_df['anomaly'] == True]
    #print("ANOMALIES:", anomalies)
    anomaly_intervals = find_anomaly_interval(anomalies['time'])
    #print(anomaly_intervals)
    return output


In [303]:
dict = {}

In [ ]:
for file in sorted(files):
    id = file.split("Np")[1].split("Nr")[0]
    if int(id) not in healthy:
        path = file
        train = pd.read_csv(path)
        test = pd.read_csv(path.replace("1N-1", "2N-2", 1))

        target = '0'

        scaler = StandardScaler()
        train[target] = scaler.fit_transform(train[[target]])
        test[target] = scaler.fit_transform(test[[target]])
        seq_size = 30  # Number of time steps to look back
        trainX, trainY = to_sequences(train[[target]], train[target], seq_size)

        testX, testY = to_sequences(test[[target]], test[target], seq_size)

        #plt.hist(trainMAE, bins=30)
        max_trainMAE = 0.5  #or Define 90% value of max as threshold.

        trainPredict = check.predict(trainX)
        trainMAE = np.mean(np.abs(trainPredict - trainX), axis=1)
        
        testPredict = check.predict(testX)
        testMAE = np.mean(np.abs(testPredict - testX), axis=1)
        
        
        anomaly_df = pd.DataFrame(test[30:])
        anomaly_df['testMAE'] = testMAE
        anomaly_df['max_trainMAE'] = max_trainMAE
        anomaly_df['anomaly'] = anomaly_df['testMAE'] > anomaly_df['max_trainMAE']
        anomaly_df[target] = test[target]
        anomaly_df['time'] = anomaly_df.index
        #Plot testMAE vs max_trainMAE
        # sns.lineplot(x=anomaly_df['Date'], y=anomaly_df['testMAE'])
        # sns.lineplot(x=anomaly_df['Date'], y=anomaly_df['max_trainMAE'])

        anomalies = anomaly_df.loc[anomaly_df['anomaly'] == True]
        anomaly_intervals = find_anomaly_interval(anomalies['time'])
        
        dict[id+"-2"] = anomaly_intervals
        
        anomaly_df = pd.DataFrame(train[30:])
        anomaly_df['trainMAE'] = trainMAE
        anomaly_df['max_trainMAE'] = max_trainMAE
        anomaly_df['anomaly'] = anomaly_df['trainMAE'] > anomaly_df['max_trainMAE']
        anomaly_df[target] = train[target]
        anomaly_df['time'] = anomaly_df.index
        #Plot testMAE vs max_trainMAE
        # sns.lineplot(x=anomaly_df['Date'], y=anomaly_df['testMAE'])
        # sns.lineplot(x=anomaly_df['Date'], y=anomaly_df['max_trainMAE'])

        anomalies = anomaly_df.loc[anomaly_df['anomaly'] == True]
        anomaly_intervals = find_anomaly_interval(anomalies['time'])
        
        dict[id+"-1"] = anomaly_intervals
        #plt.hist(testMAE, bins=30)


901/901 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
900/900 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
939/939 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
908/908 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
908/908 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
908/908 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
920/920 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
919/919 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
919/919 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
919/919 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
909/909 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
907/907 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
770/907 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

In [ ]:
dict

In [300]:
dict

{'16-2': [(1782, 1785),
  (1790, 1791),
  (1793, 1811),
  (13650, 13673),
  (13677, 13677),
  (18984, 19013),
  (21626, 21660),
  (27270, 27270),
  (27484, 27518),
  (28513, 28545)],
 '16-1': [(4543, 4543),
  (5992, 6039),
  (6041, 6042),
  (10001, 10002),
  (10009, 10012),
  (10015, 10016),
  (10018, 10027),
  (13201, 13227),
  (13456, 13456),
  (13461, 13461),
  (13469, 13473),
  (15460, 15493),
  (18634, 18634),
  (18637, 18663),
  (18913, 18913),
  (18915, 18915),
  (18918, 18918),
  (18921, 18924),
  (19235, 19235),
  (19243, 19245),
  (19247, 19250),
  (19252, 19252),
  (19782, 19785),
  (19787, 19808),
  (20938, 20959),
  (20961, 20961),
  (20963, 20963),
  (21044, 21078),
  (22240, 22269),
  (24682, 24711),
  (25013, 25045),
  (27220, 27221),
  (27223, 27224),
  (27562, 27562),
  (27572, 27572),
  (28350, 28380),
  (28580, 28580),
  (28582, 28584),
  (28586, 28600)],
 '17-2': [(236, 236),
  (238, 238),
  (247, 250),
  (252, 260),
  (262, 265),
  (10092, 10093),
  (15253, 15282)